In [ ]:
# https://www.acmicpc.net/problem/2618
"""
* BF로 풀면 2**n 이다.
* n-1 번째 까지의 이동 경로와 n번쨰 가지의 이동 경로는 연관되지 않는다.
(n번째 점이 어디있느냐에 따라서 n-1번째 까지의 경로가 완전 바뀔 수 있기 때문
즉, n까지의 최소를 구할때 n-1까지가 최소일 필요는 없다)
-> 1차원 dp로 풀이 불가

d[x][y] : 1번 경찰차가 x번째 사건을 완료했고, 2번 경찰차가 y번째 사건을 완료했을 때 최소 이동 거리
(1번 : 1,1 출발 , 2번 : n,n 출발)
=> * d[1][0], d[0][1] 초기화 하기
  * 현재 사건(now) : x+1 if x>y else y+1
    dp[now][y] = min(dp[now][y], dp[x][y] + dist(x사건 위치,now사건 위치))
    dp[x][now] = min(dp[x][now], dp[x][y] + dist(y사건 위치,now사건 위치))
  * 탐색은 bfs 방식으로 queue에 넣기
=> 여기에 순서까지 구해야 하므로 dp[x][y] = [거리 최솟값, 그때까지 사용된 경찰차 str]로 만들자
"""

"""
메모리 초과
=> deque를 사용해서 발생하는 것으로 보인다.
=> deque를 쓰지 않고 쌩으로 3중 for문 돌리는게 해결 방법이 될 것 같다.
"""

import sys
from collections import deque
input = sys.stdin.readline

n = int(input())
w = int(input())
p = [list(map(int,input().split())) for _ in range(w)]

def dist(a,b):
    return abs(a[0]-b[0]) + abs(a[1]-b[1])

# dp[x][y] : 1번 경찰차가 x번째 사건을 완료했고, 2번 경찰차가 y번째 사건을 완료했을 때 최소 이동 거리
dp = [[[1e9, ""] for _ in range(w+1)] for _ in range(w+1)]
dp[0][1] = [dist([n,n],p[0]), "2"]
dp[1][0] = [dist([1,1],p[0]), "1"]

q = deque([(0,1), (1,0)])
tmp = set()

while q:
    x, y = q.popleft()
    now = max(x,y) + 1

    if dp[now][y][0] > dp[x][y][0] + dist([1,1] if x==0 else p[x-1],p[now-1]):
        dp[now][y] = [dp[x][y][0] + dist([1,1] if x==0 else p[x-1],p[now-1]), dp[x][y][1] + "1"]

    if dp[x][now][0] > dp[x][y][0] + dist([n,n] if y==0 else p[y-1],p[now-1]):
        dp[x][now] = [dp[x][y][0] + dist([n,n] if y==0 else p[y-1],p[now-1]), dp[x][y][1] + "2"]

    if now != w:
        tmp.add((now,y))
        tmp.add((x,now))

    if not q:
        q = deque(list(tmp))
        tmp = set()

result = ''
mv = 1e9
for i in range(w):
    if dp[i][w][0] < mv:
        mv = dp[i][w][0]
        result = dp[i][w][1]
    if dp[w][i][0] < mv:
        mv = dp[w][i][0]
        result = dp[w][i][1]

print(mv)
for v in result: print(v)

In [ ]:
"""
메모리 초과 발생
deque가 문제가 아니라 경로 저장을 string으로 하기때문에 발생하는 현상이라 생각된다.
"""

import sys
from collections import deque
input = sys.stdin.readline

n = int(input())
w = int(input())
p = [list(map(int,input().split())) for _ in range(w)]

def dist(a,b):
    return abs(a[0]-b[0]) + abs(a[1]-b[1])

# dp[x][y] : 1번 경찰차가 x번째 사건을 완료했고, 2번 경찰차가 y번째 사건을 완료했을 때 최소 이동 거리
dp = [[[1e9, ""] for _ in range(w+1)] for _ in range(w+1)]
dp[0][1] = [dist([n,n],p[0]), "2"]
dp[1][0] = [dist([1,1],p[0]), "1"]

for now in range(2, w + 1):
    # x == now-1인 경우 : y = 0 ~ x-1
    x = now-1
    for y in range(0, x):
        # 1번 차
        if dp[now][y][0] > dp[x][y][0] + dist(p[x-1],p[now-1]):
            dp[now][y] = [dp[x][y][0] + dist(p[x-1],p[now-1]), dp[x][y][1] + "1"]

        # 2번 차
        if dp[x][now][0] > dp[x][y][0] + dist([n,n] if y==0 else p[y-1],p[now-1]):
            dp[x][now] = [dp[x][y][0] + dist([n,n] if y==0 else p[y-1],p[now-1]), dp[x][y][1] + "2"]

    # y == now-1인 경우 : x = 0 ~ y-1
    y = now-1
    for x in range(0,y):
        # 1번 차
        if dp[now][y][0] > dp[x][y][0] + dist([1,1] if x==0 else p[x-1],p[now-1]):
            dp[now][y] = [dp[x][y][0] + dist([1,1] if x==0 else p[x-1],p[now-1]), dp[x][y][1] + "1"]

        # 2번 차
        if dp[x][now][0] > dp[x][y][0] + dist(p[y-1],p[now-1]):
            dp[x][now] = [dp[x][y][0] + dist(p[y-1],p[now-1]), dp[x][y][1] + "2"]

result = ''
mv = 1e9
for i in range(w):
    if dp[i][w][0] < mv:
        mv = dp[i][w][0]
        result = dp[i][w][1]
    if dp[w][i][0] < mv:
        mv = dp[w][i][0]
        result = dp[w][i][1]

print(mv)
for v in result: print(v)

In [ ]:
"""
시간 초과
"""

import sys
from collections import deque
input = sys.stdin.readline

n = int(input())
w = int(input())
p = [list(map(int,input().split())) for _ in range(w)]

def dist(a,b):
    return abs(a[0]-b[0]) + abs(a[1]-b[1])

# dp[x][y] : 1번 경찰차가 x번째 사건을 완료했고, 2번 경찰차가 y번째 사건을 완료했을 때 최소 이동 거리
dp = [[[1e9, [-1,-1]] for _ in range(w+1)] for _ in range(w+1)]
dp[0][1] = [dist([n,n],p[0]), [0,0]]
dp[1][0] = [dist([1,1],p[0]), [0,0]]

for now in range(2, w + 1):
    # x == now-1인 경우 : y = 0 ~ x-1
    x = now-1
    for y in range(0, x):
        # 1번 차
        if dp[now][y][0] > dp[x][y][0] + dist(p[x-1],p[now-1]):
            dp[now][y] = [dp[x][y][0] + dist(p[x-1],p[now-1]), [x,y]]

        # 2번 차
        if dp[x][now][0] > dp[x][y][0] + dist([n,n] if y==0 else p[y-1],p[now-1]):
            dp[x][now] = [dp[x][y][0] + dist([n,n] if y==0 else p[y-1],p[now-1]), [x,y]]

    # y == now-1인 경우 : x = 0 ~ y-1
    y = now-1
    for x in range(0,y):
        # 1번 차
        if dp[now][y][0] > dp[x][y][0] + dist([1,1] if x==0 else p[x-1],p[now-1]):
            dp[now][y] = [dp[x][y][0] + dist([1,1] if x==0 else p[x-1],p[now-1]), [x,y]]

        # 2번 차
        if dp[x][now][0] > dp[x][y][0] + dist(p[y-1],p[now-1]):
            dp[x][now] = [dp[x][y][0] + dist(p[y-1],p[now-1]), [x,y]]

now = []
mv = 1e9
for i in range(w):
    if dp[i][w][0] < mv:
        mv = dp[i][w][0]
        now = [i,w]
    if dp[w][i][0] < mv:
        mv = dp[w][i][0]
        now = [w,i]

print(mv)

result = []
cnt = 0
bef = dp[now[0]][now[1]][1]
while cnt != w:
    if now[0] == bef[0]:
        result.append(2)
    else:
        result.append(1)
    now = bef
    bef = dp[now[0]][now[1]][1]

for v in reversed(result): print(v)

In [ ]:
"""
시간초과
"""

import sys
from collections import deque
input = sys.stdin.readline

n = int(input())
w = int(input())
p = [list(map(int,input().split())) for _ in range(w)]

def dist(a,b):
    return abs(a[0]-b[0]) + abs(a[1]-b[1])

# dp[x][y] : 1번 경찰차가 x번째 사건을 완료했고, 2번 경찰차가 y번째 사건을 완료했을 때 최소 이동 거리
dp = [[[1e9, [-1,-1]] for _ in range(w+1)] for _ in range(w+1)]
dp[0][1] = [dist([n,n],p[0]), [-1,-1]]
dp[1][0] = [dist([1,1],p[0]), [-1,-1]]

q = deque([(0,1), (1,0)])
tmp = set()

while q:
    x, y = q.popleft()
    now = max(x,y) + 1

    if dp[now][y][0] > dp[x][y][0] + dist([1,1] if x==0 else p[x-1],p[now-1]):
        dp[now][y] = [dp[x][y][0] + dist([1,1] if x==0 else p[x-1],p[now-1]), [x,y]]

    if dp[x][now][0] > dp[x][y][0] + dist([n,n] if y==0 else p[y-1],p[now-1]):
        dp[x][now] = [dp[x][y][0] + dist([n,n] if y==0 else p[y-1],p[now-1]), [x,y]]

    if now != w:
        tmp.add((now,y))
        tmp.add((x,now))

    if not q:
        q = deque(list(tmp))
        tmp = set()

now = []
mv = 1e9
for i in range(w):
    if dp[i][w][0] < mv:
        mv = dp[i][w][0]
        now = [i,w]
    if dp[w][i][0] < mv:
        mv = dp[w][i][0]
        now = [w,i]

print(mv)

result = []
bef = dp[now[0]][now[1]][1]
while bef != [-1,-1]:
    if now[0] == bef[0]:
        result.append(2)
    else:
        result.append(1)
    now = bef
    bef = dp[now[0]][now[1]][1]

if now == [0,1]:
    result.append(2)
else:
    result.append(1)
for v in reversed(result): print(v)

In [ ]:
"""
넓혀가면서 탐색하는게 아니라 모아오는 방식을 사용하자.
즉, dfs로 0,0에서 시작하여 마지막 사건까지 (x,y == w)까지 재귀로 들어간 다음에, 빠져나오면서 값을 더하자.

python3 시간초과 pyp3 메모리 초과
아마 순서를 출력하기 위해 dp에 다음 경로를 추가한 것이 문제였다고 생각된다.
"""

import sys
sys.setrecursionlimit(10**4)
input = sys.stdin.readline

n = int(input())
w = int(input())
p = [[]] + [list(map(int,input().split())) for _ in range(w)]
dp = [[[0,[-1,-1]] for _ in range(w+1)] for _ in range(w+1)]

def dist(a,b):
    return abs(a[0]-b[0]) + abs(a[1]-b[1])

def solve(x,y):
    if x==w or y==w: return 0 # 마지막 사건
    if dp[x][y][0]: return dp[x][y][0] # 이미 탐색된 경우
    nowcase = max(x,y) + 1

    # 각각 1, 2번 경찰차가 가는 경우
    case1 = solve(nowcase,y) + dist([1,1] if x == 0 else p[x],p[nowcase])
    case2 = solve(x,nowcase) + dist([n,n] if y == 0 else p[y],p[nowcase])

    # 그 중 최소거리 구하기
    dp[x][y] = [min(case1, case2), [nowcase,y] if case1 < case2 else [x,nowcase]]
    return dp[x][y][0]

print(solve(0,0))

nowp = [0,0]
for _ in range(w):
    nextp = dp[nowp[0]][nowp[1]][1]
    print(2 if nowp[0] == nextp[0] else 1)
    nowp = nextp

In [ ]:
# 경로찾는 방식 수정 => 정답

import sys
sys.setrecursionlimit(10**4)
input = sys.stdin.readline

n = int(input())
w = int(input())
p = [[]] + [list(map(int,input().split())) for _ in range(w)]
dp = [[0 for _ in range(w+1)] for _ in range(w+1)]

def dist(a,b):
    return abs(a[0]-b[0]) + abs(a[1]-b[1])

def solve(x,y):
    if x==w or y==w: return 0 # 마지막 사건
    if dp[x][y]: return dp[x][y] # 이미 탐색된 경우
    nowcase = max(x,y) + 1

    # 각각 1, 2번 경찰차가 가는 경우
    case1 = solve(nowcase,y) + dist([1,1] if x == 0 else p[x],p[nowcase])
    case2 = solve(x,nowcase) + dist([n,n] if y == 0 else p[y],p[nowcase])

    # 그 중 최소거리 구하기
    dp[x][y] = min(case1, case2)
    return dp[x][y]

print(solve(0,0))

x,y = 0,0
for _ in range(w):
    nowcase = max(x,y)+1
    case1 = dp[nowcase][y] + dist([1,1] if x == 0 else p[x],p[nowcase])
    case2 = dp[x][nowcase] + dist([n,n] if y == 0 else p[y],p[nowcase])
    if case1 < case2:
        print(1)
        x = nowcase
    else:
        print(2)
        y = nowcase